<h1>Chapter 15. More about Type Annotations</h1>

<h2>Overloaded Signatures</h2>

The `@typing.overload` decorator in Python allows defining multiple type signatures for a single function, indicating it can accept different argument types and return types. It’s used for type checking, but only one actual implementation is provided.

In [1]:
import functools
import operator
from collections.abc import Iterable
from typing import TypeVar, Union, overload

# Define type variables to represent any type
T = TypeVar('T')
S = TypeVar('S')


# Overload function signatures for type checking
# Overload 1: Sum with no start value defaults to returning T or int
@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...


# Overload 2: Sum with a specified start value returning T or S
@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...


def sum(it, /, start=0):
    return functools.reduce(operator.add, it, start)

<h3>Overload <code>max</code></h3>

In [2]:
from collections.abc import Callable, Iterable
from typing import Any, Protocol, TypeVar, Union, overload


class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...


T = TypeVar('T')
LT = TypeVar('LT')
DT = TypeVar('DT')

# Sentinel object for missing default argument
MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'


@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT: ...
@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT]) -> T: ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT: ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT]) -> T: ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ..., default: DT) -> Union[LT, DT]: ...


@overload
def max(
    __iterable: Iterable[T], *, key: Callable[[T], LT], default: DT
) -> Union[T, DT]: ...


def max(first, *args, key=None, default=MISSING):
    if args:
        series = args
        candidate = first
    else:
        series = iter(first)
        try:
            candidate = next(series)
        except StopIteration:
            if default is not MISSING:
                return default
            raise ValueError(EMPTY_MSG) from None
    if key is None:
        for current in series:
            if candidate < current:
                candidate = current
    else:
        candidate_key = key(candidate)
        for current in series:
            current_key = key(current)
            if candidate_key < current_key:
                candidate = current
                candidate_key = current_key
    return candidate

Arguments that implement `SupportLessThan` but without setting `key` and `default`

In [3]:
max(1, 2, -3)

2

In [4]:
max(['Go', 'Python', 'Rust'])

'Rust'

The `key` argument is set, the `default` argument is not

In [5]:
max(1, 2, -3, key=abs)

-3

In [6]:
max(['Go', 'Python', 'Rust'], key=len)

'Python'

The `default` argument is set, the `key` is not

In [7]:
max([1, 2, -3], default=0)

2

In [8]:
max([], default=None)  # returns None

The `key` and `default` arguments are set

In [9]:
max([1, 2, -3], key=abs, default=None)

-3

In [10]:
max([], key=abs, default=None)  # returns None

<h2>TypedDict</h2>

`TypedDict` is a class in Python’s `typing` module used to define dictionaries with a fixed set of keys, where each key is associated with a specific type. It allows for more precise type checking by specifying the types of individual dictionary entries, making it easier to work with structured data while catching type errors early.

In [11]:
from typing import TypedDict


class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

In [12]:
pp = BookDict(
    isbn='0134757599',
    title='Refactoring, 2e',
    authors=['Martin Fowler', 'Kent Beck'],
    pagecount=256,
)

pp

{'isbn': '0134757599',
 'title': 'Refactoring, 2e',
 'authors': ['Martin Fowler', 'Kent Beck'],
 'pagecount': 256}

In [13]:
type(pp)

dict

In [14]:
try:
    pp.title
except AttributeError as e:
    print(e.__repr__())

AttributeError("'dict' object has no attribute 'title'")


In [15]:
pp['title']

'Refactoring, 2e'

In [16]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

Using `BookDict` in the function signature

In [17]:
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'


def to_xml(book: BookDict) -> str:
    elements: list[str] = []
    for key, value in book.items():
        if isinstance(value, list):
            elements.extend(AUTHOR_ELEMENT.format(n) for n in value)
        else:
            tag = key.upper()
            elements.append(f"<{tag}>{value}</{tag}>")
    xml = '\n\t'.join(elements)
    return f"<BOOK>\n\t{xml}\n</BOOK>"


print(to_xml(pp))

<BOOK>
	<ISBN>0134757599</ISBN>
	<TITLE>Refactoring, 2e</TITLE>
	<AUTHOR>Martin Fowler</AUTHOR>
	<AUTHOR>Kent Beck</AUTHOR>
	<PAGECOUNT>256</PAGECOUNT>
</BOOK>


Function `from_json` with variable annotations that parses a `JSON` formatted string of type `str` and returns `BookDict`

In [18]:
import json


def from_json(data: str) -> BookDict:
    whatever: BookDict = json.loads(data)
    return whatever